In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Assuming you already have employee and job datasets
employee_data = pd.read_csv('employee_data.csv')
job_data = pd.read_csv('job_dataset.csv')

employee_selected_features = ['current_soft_skills', 'current_tech_stack', 'salary', 'job_recommendation1', 'job_recommendation2', 'job_recommendation3']

for feature in employee_selected_features:
    employee_data[feature] = employee_data[feature].fillna('')

# Replace empty strings or missing values in 'salary' with 0
employee_data['salary'] = employee_data['salary'].replace('', '0')

# Remove commas from the 'salary' values and convert to float
employee_data['salary'] = employee_data['salary'].replace({',': ''}, regex=True).astype(float)

# Normalize the salary values using Min-Max scaling
scaler = MinMaxScaler()
employee_data['salary'] = scaler.fit_transform(employee_data[['salary']])

employee_data['combined_features'] = employee_data['current_soft_skills'] + ' ' + employee_data['current_tech_stack'] + ' ' + employee_data['salary'].astype(str)

# Combine soft skills and tech stack into a single feature for the job dataset
job_selected_features = ['SOFT_SKILLS_REQUIRED', 'TECHSTACK_REQUIRED', 'SALARY', 'JOB_ROLE']

for feature in job_selected_features:
    job_data[feature] = job_data[feature].fillna('')

# Replace empty strings or missing values in 'SALARY' with 0
job_data['SALARY'] = job_data['SALARY'].replace('', '0')

# Remove commas from the 'SALARY' values and convert to float
job_data['SALARY'] = job_data['SALARY'].replace({',': ''}, regex=True).astype(float)

# Normalize the salary values using Min-Max scaling
job_data['SALARY'] = scaler.fit_transform(job_data[['SALARY']])

job_data['combined_features'] = job_data['SOFT_SKILLS_REQUIRED'] + ' ' + job_data['TECHSTACK_REQUIRED'] + ' ' + job_data['SALARY'].astype(str)

# Combine the datasets
full_data = pd.concat([employee_data, job_data])

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform on the combined data
full_feature_vectors = vectorizer.fit_transform(full_data['combined_features'])

# Label encode job recommendations
label_encoder = LabelEncoder()
full_data['encoded_job_recommendations'] = label_encoder.fit_transform(full_data['job_recommendation1'])
full_data['encoded_job_recommendations2'] = label_encoder.fit_transform(full_data['job_recommendation2'])
full_data['encoded_job_recommendations3'] = label_encoder.fit_transform(full_data['job_recommendation3'])
display(full_data.head(900))

emp_id    employee_name    current_domain  \
0    EMP001     Advait Reddy               R&D   
1    EMP002      Arya Kapoor               R&D   
2    EMP003     Aahana Patel               R&D   
3    EMP004  Niyati Raghavan               R&D   
4    EMP005     Aanya Bansal               R&D   
..      ...              ...               ...   
895  EMP896     Arjun Sharma  Customer Service   
896  EMP897     Aditya Menon  Customer Service   
897  EMP898     Arjun Bansal  Customer Service   
898  EMP899     Trisha Kumar  Customer Service   
899  EMP900   Varenya Bansal  Customer Service   

                              current_job  \
0    Materials Science Research Scientist   
1    Materials Science Research Scientist   
2          Strategic Partnerships Manager   
3            Space Exploration Researcher   
4            Space Exploration Researcher   
..                                    ...   
895            Multichannel Support Agent   
896       Customer Engagement Coordinator   
897            Client Billing Coordinator   
898         Customer Relations Specialist   
899         Customer Relations Specialist   

                                   current_soft_skills  \
0    Analytical Thinking, Laboratory Skills, Materi...   
1    Analytical Thinking, Laboratory Skills, Materi...   
2    Relationship Building, Negotiation, Business D...   
3         Curiosity, Analytical Thinking, Adaptability   
4         Curiosity, Analytical Thinking, Adaptability   
..                                                 ...   
895           Multichannel Communication, Adaptability   
896                          Engagement, Communication   
897             Billing Knowledge, Attention to Detail   
898                       Conflict Resolution, Empathy   
899                       Conflict Resolution, Empathy   

                                    current_tech_stack  years_of_experience  \
0    Materials Testing, Spectroscopy, R&D in Materi...                  9.0   
1    Materials Testing, Spectroscopy, R&D in Materi...                  9.0   
2        Partnership Management Tools, Market Research                  8.0   
3                      Satellite Systems, Astrophysics                  7.0   
4                      Satellite Systems, Astrophysics                  7.0   
..                                                 ...                  ...   
895               Multichannel CRM, Help Desk Software                  3.0   
896                              Engagement Tools, CRM                  3.0   
897                              Billing Software, CRM                  3.0   
898                           CRM, Communication Tools                  3.0   
899                           CRM, Communication Tools                  3.0   

       salary           job_recommendation1  \
0    1.000000       Chief Marketing Officer   
1    1.000000       Chief Marketing Officer   
2    1.000000       Chief Marketing Officer   
3    1.000000       Chief Marketing Officer   
4    1.000000       Chief Marketing Officer   
..        ...                           ...   
895  0.016949      Service Desk Coordinator   
896  0.016949      Service Desk Coordinator   
897  0.016949             Junior Accountant   
898  0.000000  Customer Servicre Supervisor   
899  0.000000  Customer Servicre Supervisor   

                      job_recommendation2  ...  \
0            Space Exploration Researcher  ...   
1            Space Exploration Researcher  ...   
2            Space Exploration Researcher  ...   
3    Materials Science Research Scientist  ...   
4    Materials Science Research Scientist  ...   
..                                    ...  ...   
895                     Junior Accountant  ...   
896                          HR Assistant  ...   
897                          HR Assistant  ...   
898         Escalation Resolution Manager  ...   
899         Escalation Resolution Manager  ...   

                                     combined_featu

In [2]:
# Split the dataset into training and testing sets
train_data, test_data = train_test_split(full_data, test_size=0.2, random_state=42)

# Normalize the continuous labels using Min-Max scaling
train_labels = scaler.fit_transform(train_data[['encoded_job_recommendations']])
test_labels = scaler.transform(test_data[['encoded_job_recommendations']])

# Instantiate the KNN regression model
k_neighbors = 4
knn_model = KNeighborsRegressor(n_neighbors=k_neighbors, weights='distance')

# Train the model on the training set
knn_model.fit(full_feature_vectors[:len(train_data)], train_labels)

# Test the model on the testing set
predicted_labels = knn_model.predict(full_feature_vectors[len(train_data):])

# Evaluate the model using Mean Squared Error
mse = mean_squared_error(test_labels, predicted_labels)
mae = mean_absolute_error(test_labels, predicted_labels)
r2 = r2_score(test_labels, predicted_labels)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R-squared: {r2}')

Mean Squared Error: 0.17419647168912264
Mean Absolute Error: 0.3334334800270819
R-squared: -0.5480117637737567


In [3]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Assuming you already have employee and job datasets
employee_data = pd.read_csv('employee_data1.csv')
job_data = pd.read_csv('job_dataset.csv')

employee_selected_features = ['current_soft_skills', 'current_tech_stack', 'salary', 'job_recommendation1', 'job_recommendation2', 'job_recommendation3']

for feature in employee_selected_features:
    employee_data[feature] = employee_data[feature].fillna('')

# Replace empty strings or missing values in 'salary' with 0
employee_data['salary'] = employee_data['salary'].replace('', '0')

# Remove commas from the 'salary' values and convert to float
employee_data['salary'] = employee_data['salary'].replace({',': ''}, regex=True).astype(float)

# Normalize the salary values using Min-Max scaling
scaler = MinMaxScaler()
employee_data['salary'] = scaler.fit_transform(employee_data[['salary']])

employee_data['combined_features'] = employee_data['current_soft_skills'] + ' ' + employee_data['current_tech_stack'] + ' ' + employee_data['salary'].astype(str)

# Combine soft skills and tech stack into a single feature for the job dataset
job_selected_features = ['SOFT_SKILLS_REQUIRED', 'TECHSTACK_REQUIRED', 'SALARY']

for feature in job_selected_features:
    job_data[feature] = job_data[feature].fillna('')

# Replace empty strings or missing values in 'SALARY' with 0
job_data['SALARY'] = job_data['SALARY'].replace('', '0')

# Remove commas from the 'SALARY' values and convert to float
job_data['SALARY'] = job_data['SALARY'].replace({',': ''}, regex=True).astype(float)

# Normalize the salary values using Min-Max scaling
job_data['SALARY'] = scaler.fit_transform(job_data[['SALARY']])

job_data['combined_features'] = job_data['SOFT_SKILLS_REQUIRED'] + ' ' + job_data['TECHSTACK_REQUIRED'] + ' ' + job_data['SALARY'].astype(str)

# Combine the datasets
full_data = pd.concat([employee_data, job_data])

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform on the combined data
full_feature_vectors = vectorizer.fit_transform(full_data['combined_features'])

# Label encode job recommendations
label_encoder = LabelEncoder()
full_data['encoded_job_recommendations'] = label_encoder.fit_transform(full_data['job_recommendation1'])
# full_data['encoded_job_recommendations2'] = label_encoder.fit_transform(full_data['job_recommendation2'])
# full_data['encoded_job_recommendations3'] = label_encoder.fit_transform(full_data['job_recommendation3'])

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(full_data, test_size=0.2, random_state=42)

# Normalize the continuous labels using Min-Max scaling
train_labels = scaler.fit_transform(train_data[['encoded_job_recommendations']])
test_labels = scaler.transform(test_data[['encoded_job_recommendations']])

# Instantiate the KNN regression model
k_neighbors = 4
knn_model = KNeighborsRegressor(n_neighbors=k_neighbors, weights='distance')

# Train the model on the training set
knn_model.fit(full_feature_vectors[:len(train_data)], train_labels)

# Test the model on the testing set
predicted_labels = knn_model.predict(full_feature_vectors[len(train_data):])

# Inverse transform the predicted labels
predicted_labels_inverse = scaler.inverse_transform(predicted_labels)

# Evaluate the model using Mean Squared Error
mse = mean_squared_error(test_labels, predicted_labels)
mae = mean_absolute_error(test_labels, predicted_labels)
r2 = r2_score(test_labels, predicted_labels)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R-squared: {r2}')

Mean Squared Error: 0.15810437970794933
Mean Absolute Error: 0.31970973782771533
R-squared: -0.44979808827754364


In [4]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Assuming you already have employee and job datasets
employee_data = pd.read_csv('employee_data1.csv')
job_data = pd.read_csv('job_dataset1.csv')
display(employee_data)

employee_selected_features = ['current_soft_skills', 'current_tech_stack', 'salary', 'job_recommendation1', 'job_recommendation2', 'job_recommendation3']

for feature in employee_selected_features:
    employee_data[feature] = employee_data[feature].fillna('')

# Replace empty strings or missing values in 'salary' with 0
employee_data['salary'] = employee_data['salary'].replace('', '0')

# Remove commas from the 'salary' values and convert to float
employee_data['salary'] = employee_data['salary'].replace({',': ''}, regex=True).astype(float)

# Normalize the salary values using Min-Max scaling
scaler = MinMaxScaler()
employee_data['salary'] = scaler.fit_transform(employee_data[['salary']])

employee_data['combined_features'] = employee_data['current_soft_skills'] + ' ' + employee_data['current_tech_stack'] + ' ' + employee_data['salary'].astype(str)

# Combine soft skills and tech stack into a single feature for the job dataset
job_selected_features = ['SOFT_SKILLS_REQUIRED', 'TECHSTACK_REQUIRED', 'SALARY']

for feature in job_selected_features:
    job_data[feature] = job_data[feature].fillna('')

# Replace empty strings or missing values in 'salary' with the mean salary
mean_salary = employee_data['salary'].mean()
employee_data['salary'].fillna(mean_salary, inplace=True)

# Remove commas from the 'SALARY' values and convert to float
job_data['SALARY'] = job_data['SALARY'].replace({',': ''}, regex=True).astype(float)

# Normalize the salary values using Min-Max scaling
job_data['SALARY'] = scaler.fit_transform(job_data[['SALARY']])

job_data['combined_features'] = job_data['SOFT_SKILLS_REQUIRED'] + ' ' + job_data['TECHSTACK_REQUIRED'] + ' ' + job_data['SALARY'].astype(str)

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Combine employee and job data into a single dataset
full_data = pd.concat([employee_data, job_data], ignore_index=True)

# Fit and transform on the combined data
full_feature_vectors = vectorizer.fit_transform(full_data['combined_features'])

# Label encode 'JOB_ROLE' in the job dataset
label_encoder = LabelEncoder()
job_data['JOB_ROLE'] = label_encoder.fit_transform(job_data['JOB_ROLE'].fillna(''))

# Combine the one-hot encoded columns with the employee_data DataFrame
employee_data = pd.concat([employee_data, job_data['JOB_ROLE']], axis=1)

# Save the DataFrame to a CSV file
employee_data.to_csv('employee_data_with_added_columns.csv', index=False)

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(employee_data, test_size=0.2, random_state=42)
# train_data.to_csv('train_data.csv', index=False)
# test_data.to_csv('test_data.csv', index=False)

# Extract labels for training and testing sets
train_labels = train_data['JOB_ROLE'].values
test_labels = test_data['JOB_ROLE'].values

# Replace NaN values in label data with a specific constant
constant_label = -1  # Choose a constant not present in your actual data
train_labels[np.isnan(train_labels)] = constant_label
test_labels[np.isnan(test_labels)] = constant_label

# Instantiate the KNN regression model
k_neighbors = 4
knn_model = KNeighborsRegressor(n_neighbors=k_neighbors, weights='distance')

# Train the model on the training set
knn_model.fit(full_feature_vectors[:len(train_data)], train_labels)

# Test the model on the testing set
predicted_labels = knn_model.predict(full_feature_vectors[len(train_data):])

# Round predicted labels to the nearest integer
predicted_labels = np.round(predicted_labels).astype(int)

# Ensure the number of samples matches
num_test_samples = len(test_labels)
predicted_labels = predicted_labels[:num_test_samples]

# Evaluate the model using Mean Squared Error
mse = mean_squared_error(test_labels, predicted_labels)
mae = mean_absolute_error(test_labels, predicted_labels)
r2 = r2_score(test_labels, predicted_labels)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R-squared: {r2}')

# Compare true and predicted labels for a sample of data
comparison_df = pd.DataFrame({'True Labels': test_labels, 'Predicted Labels': predicted_labels})
print(comparison_df.head(20))  # Adjust the number based on your dataset size

# Calculate accuracy
accuracy = (test_labels == predicted_labels).mean()

print(f'Classification Accuracy: {accuracy * 100:.2f}%')

emp_id    employee_name    current_domain  \
0    EMP001     Advait Reddy               R&D   
1    EMP002      Arya Kapoor               R&D   
2    EMP003     Aahana Patel               R&D   
3    EMP004  Niyati Raghavan               R&D   
4    EMP005     Aanya Bansal               R&D   
..      ...              ...               ...   
895  EMP896     Arjun Sharma  Customer Service   
896  EMP897     Aditya Menon  Customer Service   
897  EMP898     Arjun Bansal  Customer Service   
898  EMP899     Trisha Kumar  Customer Service   
899  EMP900   Varenya Bansal  Customer Service   

                              current_job  \
0    Materials Science Research Scientist   
1    Materials Science Research Scientist   
2          Strategic Partnerships Manager   
3            Space Exploration Researcher   
4            Space Exploration Researcher   
..                                    ...   
895            Multichannel Support Agent   
896       Customer Engagement Coordinator   
897            Client Billing Coordinator   
898         Customer Relations Specialist   
899         Customer Relations Specialist   

                                   current_soft_skills  \
0    Analytical Thinking, Laboratory Skills, Materi...   
1    Analytical Thinking, Laboratory Skills, Materi...   
2    Relationship Building, Negotiation, Business D...   
3         Curiosity, Analytical Thinking, Adaptability   
4         Curiosity, Analytical Thinking, Adaptability   
..                                                 ...   
895           Multichannel Communication, Adaptability   
896                          Engagement, Communication   
897             Billing Knowledge, Attention to Detail   
898                       Conflict Resolution, Empathy   
899                       Conflict Resolution, Empathy   

                                    current_tech_stack  years_of_experience  \
0    Materials Testing, Spectroscopy, R&D in Materi...                    9   
1    Materials Testing, Spectroscopy, R&D in Materi...                    9   
2        Partnership Management Tools, Market Research                    8   
3                      Satellite Systems, Astrophysics                    7   
4                      Satellite Systems, Astrophysics                    7   
..                                                 ...                  ...   
895               Multichannel CRM, Help Desk Software                    3   
896                              Engagement Tools, CRM                    3   
897                              Billing Software, CRM                    3   
898                           CRM, Communication Tools                    3   
899                           CRM, Communication Tools                    3   

        salary               job_recommendation1  \
0    32,50,000                Investment Analyst   
1    32,50,000                Investment Analyst   
2    32,50,000             Channel Sales Manager   
3    32,50,000              Supply Chain Analyst   
4    32,50,000              Supply Chain Analyst   
..         ...                               ...   
895   3,50,000  Multilingual Support Coordinator   
896   3,50,000    Customer Engagement Specialist   
897   3,50,000                Billing Specialist   
898   3,00,000       Customer Service Supervisor   
899   3,00,000       Customer Service Supervisor   

               job_recommendation2          job_recommendation3  
0       Demand Forecasting Analyst           Packaging Engineer  
1       Demand Forecasting Analyst           Packaging Engineer  
2            Partner Sales Manager                Brand Manager  
3         Sales Operations Manager      Billing Support Analyst  
4         Sales Operations Manager      Billing Support Analyst  
..                             ...                          ...  
895           Sales Representative          Key Account Manager  
896    Customer Engagement Manager        Onboarding Specialist 

Mean Squared Error: 31880.205555555556
Mean Absolute Error: 143.41666666666666
R-squared: -0.27641653168909674
    True Labels  Predicted Labels
0          70.0               330
1          -1.0               246
2         211.0               468
3          -1.0                60
4          39.0                60
5          -1.0                20
6         299.0                -1
7         110.0                71
8          72.0               106
9          86.0               154
10        139.0                84
11         -1.0               186
12         -1.0                -1
13        120.0               154
14         -1.0                84
15         -1.0               186
16        168.0                -1
17        422.0               193
18        447.0               193
19         67.0               209
Classification Accuracy: 7.22%


In [5]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Assuming you already have employee and job datasets
employee_data = pd.read_csv('employee_data1.csv')
job_data = pd.read_csv('job_dataset1.csv')
display(employee_data)

employee_selected_features = ['current_soft_skills', 'current_tech_stack', 'salary', 'job_recommendation1', 'job_recommendation2', 'job_recommendation3']

for feature in employee_selected_features:
    employee_data[feature] = employee_data[feature].fillna('')

# Replace empty strings or missing values in 'salary' with 0
employee_data['salary'] = employee_data['salary'].replace('', '0')

# Remove commas from the 'salary' values and convert to float
employee_data['salary'] = employee_data['salary'].replace({',': ''}, regex=True).astype(float)

# Normalize the salary values using Min-Max scaling
scaler = MinMaxScaler()
employee_data['salary'] = scaler.fit_transform(employee_data[['salary']])

employee_data['combined_features'] = employee_data['current_soft_skills'] + ' ' + employee_data['current_tech_stack'] + ' ' + employee_data['salary'].astype(str)

# Combine soft skills and tech stack into a single feature for the job dataset
job_selected_features = ['SOFT_SKILLS_REQUIRED', 'TECHSTACK_REQUIRED', 'SALARY']

for feature in job_selected_features:
    job_data[feature] = job_data[feature].fillna('')

# Replace empty strings or missing values in 'salary' with the mean salary
mean_salary = employee_data['salary'].mean()
employee_data['salary'].fillna(mean_salary, inplace=True)

# Remove commas from the 'SALARY' values and convert to float
job_data['SALARY'] = job_data['SALARY'].replace({',': ''}, regex=True).astype(float)

# Normalize the salary values using Min-Max scaling
job_data['SALARY'] = scaler.fit_transform(job_data[['SALARY']])

job_data['combined_features'] = job_data['SOFT_SKILLS_REQUIRED'] + ' ' + job_data['TECHSTACK_REQUIRED'] + ' ' + job_data['SALARY'].astype(str)

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Combine employee and job data into a single dataset
full_data = pd.concat([employee_data, job_data], ignore_index=True)

# Fit and transform on the combined data
full_feature_vectors = vectorizer.fit_transform(full_data['combined_features'])

# Label encode 'JOB_ROLE' in the job dataset
label_encoder = LabelEncoder()
job_data['JOB_ROLE'] = label_encoder.fit_transform(job_data['JOB_ROLE'].fillna(''))

# Combine the one-hot encoded columns with the employee_data DataFrame
employee_data = pd.concat([employee_data, job_data['JOB_ROLE']], axis=1)

# Save the DataFrame to a CSV file
employee_data.to_csv('employee_data_with_added_columns.csv', index=False)

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(employee_data, test_size=0.2, random_state=42)
# train_data.to_csv('train_data.csv', index=False)
# test_data.to_csv('test_data.csv', index=False)

# Extract labels for training and testing sets
train_labels = train_data['JOB_ROLE'].values
test_labels = test_data['JOB_ROLE'].values

# Replace NaN values in label data with a specific constant
constant_label = -1  # Choose a constant not present in your actual data
train_labels[np.isnan(train_labels)] = constant_label
test_labels[np.isnan(test_labels)] = constant_label

# Instantiate the KNN regression model
k_neighbors = 4
knn_model = KNeighborsRegressor(n_neighbors=k_neighbors, weights='distance')

# Train the model on the training set
knn_model.fit(full_feature_vectors[:len(train_data)], train_labels)

# Test the model on the testing set
predicted_labels = knn_model.predict(full_feature_vectors[len(train_data):])

# Round predicted labels to the nearest integer
predicted_labels = np.round(predicted_labels).astype(int)

# Ensure the number of samples matches
num_test_samples = len(test_labels)
predicted_labels = predicted_labels[:num_test_samples]

# Evaluate the model using Mean Squared Error
mse = mean_squared_error(test_labels, predicted_labels)
mae = mean_absolute_error(test_labels, predicted_labels)
r2 = r2_score(test_labels, predicted_labels)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R-squared: {r2}')

# Compare true and predicted labels for a sample of data
comparison_df = pd.DataFrame({'True Labels': test_labels, 'Predicted Labels': predicted_labels})
print(comparison_df.head(20))  # Adjust the number based on your dataset size

# Calculate accuracy
accuracy = (test_labels == predicted_labels).mean()

print(f'Classification Accuracy: {accuracy * 100:.2f}%')

emp_id    employee_name    current_domain  \
0    EMP001     Advait Reddy               R&D   
1    EMP002      Arya Kapoor               R&D   
2    EMP003     Aahana Patel               R&D   
3    EMP004  Niyati Raghavan               R&D   
4    EMP005     Aanya Bansal               R&D   
..      ...              ...               ...   
895  EMP896     Arjun Sharma  Customer Service   
896  EMP897     Aditya Menon  Customer Service   
897  EMP898     Arjun Bansal  Customer Service   
898  EMP899     Trisha Kumar  Customer Service   
899  EMP900   Varenya Bansal  Customer Service   

                              current_job  \
0    Materials Science Research Scientist   
1    Materials Science Research Scientist   
2          Strategic Partnerships Manager   
3            Space Exploration Researcher   
4            Space Exploration Researcher   
..                                    ...   
895            Multichannel Support Agent   
896       Customer Engagement Coordinator   
897            Client Billing Coordinator   
898         Customer Relations Specialist   
899         Customer Relations Specialist   

                                   current_soft_skills  \
0    Analytical Thinking, Laboratory Skills, Materi...   
1    Analytical Thinking, Laboratory Skills, Materi...   
2    Relationship Building, Negotiation, Business D...   
3         Curiosity, Analytical Thinking, Adaptability   
4         Curiosity, Analytical Thinking, Adaptability   
..                                                 ...   
895           Multichannel Communication, Adaptability   
896                          Engagement, Communication   
897             Billing Knowledge, Attention to Detail   
898                       Conflict Resolution, Empathy   
899                       Conflict Resolution, Empathy   

                                    current_tech_stack  years_of_experience  \
0    Materials Testing, Spectroscopy, R&D in Materi...                    9   
1    Materials Testing, Spectroscopy, R&D in Materi...                    9   
2        Partnership Management Tools, Market Research                    8   
3                      Satellite Systems, Astrophysics                    7   
4                      Satellite Systems, Astrophysics                    7   
..                                                 ...                  ...   
895               Multichannel CRM, Help Desk Software                    3   
896                              Engagement Tools, CRM                    3   
897                              Billing Software, CRM                    3   
898                           CRM, Communication Tools                    3   
899                           CRM, Communication Tools                    3   

        salary               job_recommendation1  \
0    32,50,000                Investment Analyst   
1    32,50,000                Investment Analyst   
2    32,50,000             Channel Sales Manager   
3    32,50,000              Supply Chain Analyst   
4    32,50,000              Supply Chain Analyst   
..         ...                               ...   
895   3,50,000  Multilingual Support Coordinator   
896   3,50,000    Customer Engagement Specialist   
897   3,50,000                Billing Specialist   
898   3,00,000       Customer Service Supervisor   
899   3,00,000       Customer Service Supervisor   

               job_recommendation2          job_recommendation3  
0       Demand Forecasting Analyst           Packaging Engineer  
1       Demand Forecasting Analyst           Packaging Engineer  
2            Partner Sales Manager                Brand Manager  
3         Sales Operations Manager      Billing Support Analyst  
4         Sales Operations Manager      Billing Support Analyst  
..                             ...                          ...  
895           Sales Representative          Key Account Manager  
896    Customer Engagement Manager        Onboarding Specialist 

Mean Squared Error: 31880.205555555556
Mean Absolute Error: 143.41666666666666
R-squared: -0.27641653168909674
    True Labels  Predicted Labels
0          70.0               330
1          -1.0               246
2         211.0               468
3          -1.0                60
4          39.0                60
5          -1.0                20
6         299.0                -1
7         110.0                71
8          72.0               106
9          86.0               154
10        139.0                84
11         -1.0               186
12         -1.0                -1
13        120.0               154
14         -1.0                84
15         -1.0               186
16        168.0                -1
17        422.0               193
18        447.0               193
19         67.0               209
Classification Accuracy: 7.22%
